In [1]:
import os
import time
import yaml
import math
import pandas as pd
import numpy as np

In [2]:
from data.excel2pd import data_load
from data.preprocessing import data_preprocessing
from data.train_test_split import data2Window, X_Y2train_test, X_Y_from_data
from data.CustomDataset import CustomDataset

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from model.model import seq2seq

In [4]:
# torch version, gpu 사용 여부, 이름 체크
print("torch version>>", torch.__version__)
print("GPU 사용 유무>>", torch.cuda.is_available())
if torch.cuda.is_available() :
    print("GPU 이름>> ", torch.cuda.get_device_name(0))
    device = "cuda:0" 
else :
    device = "cpu"
print(device)

torch version>> 1.13.1
GPU 사용 유무>> True
GPU 이름>>  NVIDIA GeForce RTX 2080
cuda:0


In [5]:
f = open("Settings.yaml", 'r')
setup = yaml.load(f, Loader=yaml.FullLoader)

data = data_load(11)
scaler, data = data_preprocessing(data, setup['data_options'])

X, Y = X_Y_from_data(data)

X_train, Y_train, X_test, Y_test = X_Y2train_test(X, Y,
                                                  setup['input_window'],
                                                  setup['output_window'],
                                                  setup['stride'])

In [6]:
dataset = CustomDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size = setup['batch'], shuffle = False)

In [7]:
model = seq2seq(input_window = setup['input_window'], 
output_window = setup['output_window'], 
input_size = 3, 
hidden_size = setup['LSTM'][0], 
num_layers = 1, 
dropout = setup['dropout']).to(device)

In [8]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [10]:
model.apply(init_weights)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 680 trainable parameters


In [11]:
model.train()

seq2seq(
  (encoder): Encoder(
    (lstm): LSTM(3, 7, batch_first=True)
  )
  (decoder): Decoder(
    (lstm): LSTM(3, 7, batch_first=True)
    (output_layer): Linear(in_features=7, out_features=1, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
)

In [12]:
from model.train import train

model = train(device = device, model = model, 
dataloader = dataloader, 
learning_rate = setup['learning_rate'], 
epoch = setup['epoch'],
output_window = setup['output_window'])

  0%|          | 0/10 [00:00<?, ?it/s]

Non-Teacher forcing
model output shape :  torch.Size([256, 6])
label shape :  torch.Size([256, 6])
loss :  tensor(123.4576, device='cuda:0', grad_fn=<MeanBackward0>)


c:\ProgramData\Anaconda3\envs\torch\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: Error detected in CudnnRnnBackward0. Traceback of forward call that caused the error:
  File "c:\ProgramData\Anaconda3\envs\torch\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\ProgramData\Anaconda3\envs\torch\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\ProgramData\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\ProgramData\Anaconda3\envs\torch\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
    app.start()
  File "c:\ProgramData\Anaconda3\envs\torch\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
    self.io_loop.start()
  File "c:\ProgramData\Anaconda3\envs\torch\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "c:

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [256, 6, 3]], which is output 0 of AsStridedBackward0, is at version 5; expected version 4 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!